<a href="https://colab.research.google.com/github/davidarvai/MRI-Image-Viewer/blob/main/UnetKod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import csv

# Paths
pathSection = '/content/drive/My Drive/Allamvizsga/MRI_felvetel/Teszt'
pathSection1To7 = '/content/drive/My Drive/Allamvizsga/Teszt_folder/hg000.csv'
output_text_file = '/content/drive/My Drive/Allamvizsga/Eredmeny/output_metrics_unet.txt'
output_csv_file = '/content/drive/My Drive/Allamvizsga/Eredmeny/output_unet.csv'

# CSV Header
header = ['volumeName', 'tumorType', 'truePositive', 'trueNegative', 'falsePositive', 'falseNegative',
          'truePositiveRate', 'trueNegativeRate', 'positivePredictiveValue',
          'negativePredictiveValue', 'accuracy', 'diceScore']

# Save header to CSV
with open(output_csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

# Define U-Net Model
def unet_model(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    up1 = UpSampling2D(size=(2, 2))(conv3)
    up1 = Concatenate()([up1, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up2 = UpSampling2D(size=(2, 2))(conv4)
    up2 = Concatenate()([up2, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = Conv2D(4, 1, activation='sigmoid')(conv5)  # 4 output channels for each tumor type
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Data Loader Placeholder
def load_image_data_for_each_file(file_path):
    # Replace this placeholder with actual image loading logic
    X = np.random.rand(5, 256, 256, 1)  # Example batch of 5 images
    Y = np.random.randint(0, 2, (5, 256, 256, 4))  # Example batch of 4-channel masks
    return X, Y

# Metrics Calculation
def calculate_metrics(y_true, y_pred, tumorType, volumeName):
    y_true = y_true.flatten()
    y_pred = (y_pred.flatten() > 0.5).astype(int)

    CM = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = CM.ravel() if CM.size == 4 else (0, 0, 0, 0)

    TPR = round(TP / (TP + FN), 3) if TP + FN > 0 else 0
    TNR = round(TN / (TN + FP), 3) if TN + FP > 0 else 0
    PPV = round(TP / (TP + FP), 3) if TP + FP > 0 else 0
    NPV = round(TN / (TN + FN), 3) if TN + FN > 0 else 0
    ACC = round((TP + TN) / (TP + FP + FN + TN), 3)
    DS = round((2 * TP) / ((2 * TP) + FP + FN), 3) if (2 * TP) + FP + FN > 0 else 0

    # Save metrics to CSV
    with open(output_csv_file, 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([volumeName, tumorType, TP, TN, FP, FN, TPR, TNR, PPV, NPV, ACC, DS])

    # Save metrics to text file
    with open(output_text_file, 'a') as txt_file:
        txt_file.write(f"\n=== {tumorType} Metrics for file: {volumeName} ===\n")
        txt_file.write(f"Confusion Matrix:\n{CM}\n")
        txt_file.write(f"True positive (TP): {TP}\n")
        txt_file.write(f"True negative (TN): {TN}\n")
        txt_file.write(f"False positive (FP): {FP}\n")
        txt_file.write(f"False negative (FN): {FN}\n")
        txt_file.write(f"True positive rate (TPR): {TPR:.3f}\n")
        txt_file.write(f"True negative rate (TNR): {TNR:.3f}\n")
        txt_file.write(f"Positive predictive value (PPV): {PPV:.3f}\n")
        txt_file.write(f"Negative predictive value (NPV): {NPV:.3f}\n")
        txt_file.write(f"Accuracy (ACC): {ACC:.3f}\n")
        txt_file.write(f"Dice score (DS): {DS:.3f}\n\n")

# Load Data
X_sample, Y_sample = load_image_data_for_each_file(pathSection1To7)

# Initialize Model
unet = unet_model()

# Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

augmented_data = datagen.flow(X_sample, Y_sample, batch_size=8)

# Training Callbacks
callbacks = [
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_unet_model.keras', save_best_only=True, monitor='loss', verbose=1)  # Updated file extension
]

# Train Model
history = unet.fit(
    augmented_data,
    epochs=50,
    steps_per_epoch=10,  # Adjust as per dataset size
    callbacks=callbacks,
    verbose=1
)

# Process Each Test File
tumor_types = ["Whole Tumor", "Edema", "Tumor Core", "Enhancing Core"]

for filename in os.listdir(pathSection):
    file_path = os.path.join(pathSection, filename)
    if os.path.isfile(file_path):
        # Load data
        X, Y = load_image_data_for_each_file(file_path)
        y_pred = unet.predict(X)

        volumeName = os.path.basename(file_path)

        for idx, tumorType in enumerate(tumor_types):
            calculate_metrics(Y[..., idx], y_pred[..., idx], tumorType, volumeName)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/10 ━━━━━━━━━━━━━━━━━━━━ 4:46 32s/step - accuracy: 0.1734 - loss: 0.6932
Epoch 1: loss improved from inf to 0.69318, saving model to best_unet_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.1734 - loss: 0.6932 - learning_rate: 0.0010
Epoch 2/50


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


 1/10 ━━━━━━━━━━━━━━━━━━━━ 3:23 23s/step - accuracy: 0.2408 - loss: 0.6932
Epoch 2: loss improved from 0.69318 to 0.69316, saving model to best_unet_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - accuracy: 0.2408 - loss: 0.6932 - learning_rate: 0.0010
Epoch 3/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 3:41 25s/step - accuracy: 0.1585 - loss: 0.6932
Epoch 3: loss improved from 0.69316 to 0.69315, saving model to best_unet_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.1585 - loss: 0.6932 - learning_rate: 0.0010
Epoch 4/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 6:07 41s/step - accuracy: 0.3813 - loss: 0.6931
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: loss improved from 0.69315 to 0.69315, saving model to best_unet_model.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.3813 - loss: 0.6931 - learning_rate: 0.0010
Epoch 5/50
 1/10 ━━━━━━━━━━━━━━━━━━━━ 3:20 22s/step - accuracy: 0.4618 - loss: 0.6931
Epoch 5: loss did not improve from 